In [64]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [65]:
import pandas as pd
import numpy as np
import os, sys
import argparse
from datetime import datetime
from collections import Counter

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score, cross_validate

##Import confusion_matrix.py module
from src.plot_confusion_matrix import plot_cm

In [66]:
#https://stackoverflow.com/questions/50204556/tkinter-tclerror-couldnt-connect-to-display-localhost10-0-when-using-wordc
matplotlib.use('Agg')

In [67]:
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/total.test.bed.gz'
output_path='/pod/2/li-lab/Ziwei/Nanopore/daily/2021-10-24'
confusion_matrix='confusion_matrix.png'
class_report='class_report.csv'

In [68]:
if input_path.endswith(".gz"):
    df=pd.read_csv(input_path, compression='gzip', sep='\t')
    print("Data is loading!")

Data is loading!


In [69]:
df_feature = df.loc[:,['5hmC_prob','5mC_prob','5C_prob']].values
df_class = df.loc[:,['label']].values
df_class = np.squeeze(df_class) #Convert the label into 1d-array

X = df_feature
y = df_class

#Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)
print("Spliting is done!")
print("Before the pipeline:\n y_train:{},\n y_test: {}".format(Counter(y_train), Counter(y_test)))

# Define cross-validation fold
stratified_kfold = StratifiedKFold(n_splits = 3, random_state = 42, shuffle = True)
    
model_best = imbpipeline(steps=[('o', SMOTE(random_state=42)),
                                ('m', RandomForestClassifier(max_depth=5, min_samples_leaf=2,n_estimators=80, random_state=42))])

Spliting is done!
Before the pipeline:
 y_train:Counter({0: 799, 1: 799, 2: 399}),
 y_test: Counter({0: 200, 1: 200, 2: 100})


In [74]:
model_best.fit(X_train, y_train)

Pipeline(steps=[('o', SMOTE(random_state=42)),
                ('m',
                 RandomForestClassifier(max_depth=5, min_samples_leaf=2,
                                        n_estimators=80, random_state=42))])

In [75]:
scoring = ('accuracy', 'recall_macro', 'f1_macro')
scores = cross_validate(model_best, X_train, y_train, scoring=scoring, cv=stratified_kfold, n_jobs=-1)

#Save the scoring matrix to dataframe
test = pd.DataFrame.from_dict(scores)
test.to_csv(os.path.join(output_path, class_report), header = True, index= None, sep = ',', float_format='%.4f')

In [76]:
####Traing dataset
scores['test_accuracy'].mean()

0.9223862960705066

In [77]:
y_true, y_pred = y_test, model_best.predict(X_test)
test_score = model_best.score(X_test, y_test)
test_score

0.882

In [50]:
target_class = ['5C', '5mC', '5hmC']
report = classification_report(y_test, y_pred, target_names=target_class, output_dict=True)
report = pd.DataFrame(report).transpose()
report

,precision,recall,f1-score,support
5C,0.929293,0.920,0.924623,200.000
5mC,0.957219,0.895,0.925065,200.000
5hmC,0.678261,0.780,0.725581,100.000
accuracy,0.882000,0.882,0.882000,0.882
macro avg,0.854924,0.865,0.858423,500.000
weighted avg,0.890257,0.882,0.884991,500.000
